# We seem to get different results with exact pml than with NG pml

I could have an error in my implementation, but also with lossy materials things seem to react to much smaller alphas than I'd have anticipated.  I'm trying to check what ngpml is doing here more precisely.

In [1]:
import ngsolve as ng
from ngsolve.webgui import Draw
from netgen.geom2d import SplineGeometry
import matplotlib.pyplot as plt

importing NGSolve-6.2.2203-56-g47f483033


In [2]:
geo = SplineGeometry()
geo.AddCircle( (0,0), 2, leftdomain=2, bc="outerbnd")
geo.AddCircle( (0,0), 1, leftdomain=1, rightdomain=2, bc="innerbnd")
geo.SetMaterial(1, "inner")
geo.SetMaterial(2, "pmlregion")
mesh = ng.Mesh(geo.GenerateMesh (maxh=0.08))
mesh.Curve(3)

f = ng.exp(-20**2*((ng.x-0.3)*(ng.x-0.3)+ng.y*ng.y))
Draw(f, mesh, 'f')

WebGuiWidget(value={'ngsolve_version': '6.2.2203-56-g47f483033', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'd…

BaseWebGuiScene

In [3]:
PML = ng.pml.Radial(rad=1,alpha=1j,origin=(0,0))

In [3]:
mesh.SetPML(ng.pml.Radial(rad=1,alpha=1j,origin=(0,0)), "pmlregion")


In [9]:
fes = ng.H1(mesh, order=4, complex=True)
u = fes.TrialFunction()
v = fes.TestFunction()

omega = 10

a = ng.BilinearForm(fes)
a += ng.grad(u)*ng.grad(v)*ng.dx - omega**2*u*v*ng.dx
a += -1j*omega*u*v*ng.ds("outerbnd")

b = ng.LinearForm(fes)
b += f * v * ng.dx

a.Assemble()
b.Assemble()

gfu = ng.GridFunction(fes)
# gfu.vec.data = 1e3*a.mat.Inverse() * b.vec
gfu.vec[:] = 0
gfu.vec[1] = 1+2j

Draw(gfu)

WebGuiWidget(value={'ngsolve_version': '6.2.2203-56-g47f483033', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'd…

BaseWebGuiScene

In [5]:
Draw(PML.Det_CF, mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2203-56-g47f483033', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'd…

BaseWebGuiScene

In [8]:
Rout = 2
R = 1
d = 1
alpha = 1

import ngsolve as ng
# Set complex trafo
rh = ng.sqrt(ng.x**2 + ng.y**2)  # radial variable
r = rh + 1/(d+1) * alpha * 1j * (rh - R) ** (d+1)
drdrh = 1 + alpha*1j * (rh - R) ** d

x = r * ng.x / rh
y = r * ng.y / rh

trafo = ng.CoefficientFunction((x, y))

# derivatives and determinant

dxdxh = drdrh * (ng.x / rh) ** 2 + r * (ng.y ** 2 / rh ** 3)
dxdyh = drdrh * (ng.y * ng. x / rh ** 2) - \
    r * (ng.y * ng.x / rh ** 3)

dydyh = drdrh * (ng.y / rh) ** 2 + r * (ng.x ** 2 / rh ** 3)
dydxh = dxdyh

J = ng.CoefficientFunction((dxdxh, dxdyh, dydxh, dydyh),
                           dims=(2, 2))

custom_pml = ng.pml.Custom(trafo, J)



In [10]:
Draw((custom_pml.PML_CF[1]**2 + custom_pml.PML_CF[0]**2)**.5, mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2203-56-g47f483033', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'd…

BaseWebGuiScene

In [11]:
Draw(custom_pml.Det_CF, mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2203-56-g47f483033', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'd…

BaseWebGuiScene